<a href="https://colab.research.google.com/github/rishiguptha54/FlightOpsPro_45/blob/main/Flight_delays.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
usdot_flight_delays_path = kagglehub.dataset_download('usdot/flight-delays')

print('Data source import complete.')


Data source import complete.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix , classification_report
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('/kaggle/input/flight-delays/flights.csv')

In [4]:
data.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')

In [6]:
data.shape

(5819079, 31)

In [7]:
data.isnull().sum()

,0
YEAR,0
MONTH,0
DAY,0
DAY_OF_WEEK,0
AIRLINE,0
FLIGHT_NUMBER,0
TAIL_NUMBER,14721
ORIGIN_AIRPORT,0
DESTINATION_AIRPORT,0
SCHEDULED_DEPARTURE,0


In [8]:
5729195/5819079 # 98% of the data will be deleted

0.9845535693878705

In [9]:
data_test = data.copy()

In [10]:
data_test.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
data_test.shape

(5819079, 31)

In [12]:
data_test = data_test.dropna(subset = ['CANCELLATION_REASON']) # delete rows based on specific column
data_test.shape

(89884, 31)

In [13]:
data_test.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
32,2015,1,1,4,AS,136,N431AS,ANC,SEA,135,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
42,2015,1,1,4,AA,2459,N3BDAA,PHX,DFW,200,...,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN
68,2015,1,1,4,OO,5254,N746SK,MAF,IAH,510,...,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN
82,2015,1,1,4,MQ,2859,N660MQ,SGF,DFW,525,...,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN
90,2015,1,1,4,OO,5460,N583SW,RDD,SFO,530,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN


In [14]:
data_test.isnull().sum()

,0
YEAR,0
MONTH,0
DAY,0
DAY_OF_WEEK,0
AIRLINE,0
FLIGHT_NUMBER,0
TAIL_NUMBER,14721
ORIGIN_AIRPORT,0
DESTINATION_AIRPORT,0
SCHEDULED_DEPARTURE,0


In [ ]:
# We cannot delete all rows of Candellation reason because we want to focus on arrival delay

In [15]:
data.shape

(5819079, 31)

In [16]:
data.isnull().sum()

,0
YEAR,0
MONTH,0
DAY,0
DAY_OF_WEEK,0
AIRLINE,0
FLIGHT_NUMBER,0
TAIL_NUMBER,14721
ORIGIN_AIRPORT,0
DESTINATION_AIRPORT,0
SCHEDULED_DEPARTURE,0


In [17]:
# We can try to see if we dropped the nulls in the delaye types columns
data_test2 = data.dropna(subset = ['AIR_SYSTEM_DELAY'])
data_test2.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
27,2015,1,1,4,NK,597,N528NK,MSP,FLL,115,...,607.0,25.0,0,0,NaN,25.0,0.0,0.0,0.0,0.0
30,2015,1,1,4,NK,168,N629NK,PHX,ORD,125,...,632.0,43.0,0,0,NaN,43.0,0.0,0.0,0.0,0.0
35,2015,1,1,4,HA,17,N389HA,LAS,HNL,145,...,610.0,15.0,0,0,NaN,0.0,0.0,15.0,0.0,0.0
50,2015,1,1,4,B6,1030,N239JB,BQN,MCO,307,...,520.0,20.0,0,0,NaN,20.0,0.0,0.0,0.0,0.0
52,2015,1,1,4,B6,2134,N307JB,SJU,MCO,400,...,730.0,85.0,0,0,NaN,0.0,0.0,85.0,0.0,0.0


In [18]:
data_test2.shape

(1063439, 31)

In [19]:
data_test2.isnull().sum()

,0
YEAR,0
MONTH,0
DAY,0
DAY_OF_WEEK,0
AIRLINE,0
FLIGHT_NUMBER,0
TAIL_NUMBER,0
ORIGIN_AIRPORT,0
DESTINATION_AIRPORT,0
SCHEDULED_DEPARTURE,0


We now can delete CANCELLATION_REASON column because its the problem

In [20]:
data_test2 = data_test2.drop(columns = ['CANCELLATION_REASON'], axis = 1)
data_test2.isnull().sum()

,0
YEAR,0
MONTH,0
DAY,0
DAY_OF_WEEK,0
AIRLINE,0
FLIGHT_NUMBER,0
TAIL_NUMBER,0
ORIGIN_AIRPORT,0
DESTINATION_AIRPORT,0
SCHEDULED_DEPARTURE,0


In [21]:
data_test2.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
27,2015,1,1,4,NK,597,N528NK,MSP,FLL,115,...,542,607.0,25.0,0,0,25.0,0.0,0.0,0.0,0.0
30,2015,1,1,4,NK,168,N629NK,PHX,ORD,125,...,549,632.0,43.0,0,0,43.0,0.0,0.0,0.0,0.0
35,2015,1,1,4,HA,17,N389HA,LAS,HNL,145,...,555,610.0,15.0,0,0,0.0,0.0,15.0,0.0,0.0
50,2015,1,1,4,B6,1030,N239JB,BQN,MCO,307,...,500,520.0,20.0,0,0,20.0,0.0,0.0,0.0,0.0
52,2015,1,1,4,B6,2134,N307JB,SJU,MCO,400,...,605,730.0,85.0,0,0,0.0,0.0,85.0,0.0,0.0


In [22]:
data_test2['ORIGIN_AIRPORT'].unique(),data_test2['DESTINATION_AIRPORT'].unique()

(array(['MSP', 'PHX', 'LAS', 'BQN', 'SJU', 'DFW', 'PDX', 'IAH', 'DEN',
        'SFO', 'JFK', 'SMF', 'MFR', 'MKE', 'CLE', 'BFL', 'OMA', 'SEA',
        'BUF', 'ORD', 'ONT', 'PIT', 'ABQ', 'BWI', 'LGA', 'OKC', 'RNO',
        'FLL', 'SAV', 'CLT', 'ERI', 'ATL', 'ACY', 'DTW', 'BOI', 'CLD',
        'FAT', 'PSP', 'PBI', 'RDU', 'TUS', 'EWR', 'MIA', 'MLU', 'CMH',
        'LAW', 'CHS', 'DCA', 'DRO', 'CAE', 'MCO', 'IAD', 'PHL', 'AUS',
        'TPA', 'SLC', 'BDL', 'MSN', 'LNK', 'HNL', 'BUR', 'SAN', 'BOS',
        'SAT', 'IND', 'FSD', 'LIH', 'GUM', 'ASE', 'MHK', 'OAK', 'ORF',
        'LAX', 'RAP', 'BRD', 'SGF', 'COU', 'JNU', 'SHV', 'MEM', 'RIC',
        'TYS', 'SBP', 'VEL', 'TUL', 'JAC', 'MDW', 'SAF', 'PIA', 'MCI',
        'YUM', 'FCA', 'AVL', 'HDN', 'ELP', 'ITO', 'CRW', 'DSM', 'RSW',
        'STL', 'LBB', 'SBA', 'GEG', 'CMI', 'MSO', 'MLI', 'SNA', 'CWA',
        'EVV', 'DAY', 'MYR', 'LIT', 'STX', 'CID', 'GRR', 'SDF', 'COS',
        'OGG', 'HOU', 'CDC', 'SJC', 'ROC', 'MSY', 'AGS', 'JAX', 'HRL',
      

Because both has no important information we can drop both columns beside that we have other useless columns we can get rid off.

In [23]:
data_test2.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'AIR_SYSTEM_DELAY',
       'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
       'WEATHER_DELAY'],
      dtype='object')

In [24]:
columns_to_delete = ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']
data_test2 = data_test2.drop(columns = columns_to_delete, axis = 1)

In [25]:
data_test2.head()

,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,...,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
27,115,127.0,12.0,14.0,141.0,207.0,220.0,166.0,1487,527.0,...,542,607.0,25.0,0,0,25.0,0.0,0.0,0.0,0.0
30,125,237.0,72.0,9.0,246.0,204.0,175.0,156.0,1440,622.0,...,549,632.0,43.0,0,0,43.0,0.0,0.0,0.0,0.0
35,145,145.0,0.0,16.0,201.0,370.0,385.0,361.0,2762,602.0,...,555,610.0,15.0,0,0,0.0,0.0,15.0,0.0,0.0
50,307,304.0,-3.0,25.0,329.0,173.0,196.0,160.0,1129,509.0,...,500,520.0,20.0,0,0,20.0,0.0,0.0,0.0,0.0
52,400,535.0,95.0,9.0,544.0,185.0,175.0,163.0,1189,727.0,...,605,730.0,85.0,0,0,0.0,0.0,85.0,0.0,0.0


In [26]:
data_test2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1063439 entries, 27 to 5819071
Data columns (total 21 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   SCHEDULED_DEPARTURE  1063439 non-null  int64  
 1   DEPARTURE_TIME       1063439 non-null  float64
 2   DEPARTURE_DELAY      1063439 non-null  float64
 3   TAXI_OUT             1063439 non-null  float64
 4   WHEELS_OFF           1063439 non-null  float64
 5   SCHEDULED_TIME       1063439 non-null  float64
 6   ELAPSED_TIME         1063439 non-null  float64
 7   AIR_TIME             1063439 non-null  float64
 8   DISTANCE             1063439 non-null  int64  
 9   WHEELS_ON            1063439 non-null  float64
 10  TAXI_IN              1063439 non-null  float64
 11  SCHEDULED_ARRIVAL    1063439 non-null  int64  
 12  ARRIVAL_TIME         1063439 non-null  float64
 13  ARRIVAL_DELAY        1063439 non-null  float64
 14  DIVERTED             1063439 non-null  int64  
 15  CA

In [28]:
data_test2 = data_test2.drop(columns = ['DIVERTED','CANCELLED'], axis = 1)


In [30]:
data_test2.head()

,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
27,115,127.0,12.0,14.0,141.0,207.0,220.0,166.0,1487,527.0,40.0,542,607.0,25.0,25.0,0.0,0.0,0.0,0.0
30,125,237.0,72.0,9.0,246.0,204.0,175.0,156.0,1440,622.0,10.0,549,632.0,43.0,43.0,0.0,0.0,0.0,0.0
35,145,145.0,0.0,16.0,201.0,370.0,385.0,361.0,2762,602.0,8.0,555,610.0,15.0,0.0,0.0,15.0,0.0,0.0
50,307,304.0,-3.0,25.0,329.0,173.0,196.0,160.0,1129,509.0,11.0,500,520.0,20.0,20.0,0.0,0.0,0.0,0.0
52,400,535.0,95.0,9.0,544.0,185.0,175.0,163.0,1189,727.0,3.0,605,730.0,85.0,0.0,0.0,85.0,0.0,0.0


In [31]:
scaler = StandardScaler()

In [32]:
data_test3 = scaler.fit_transform(data_test2) # numpy array will be formed
data_final = pd.DataFrame(data_test3,columns = data_test2.columns)

In [33]:
data_final.head()

,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,-2.970484,-2.859507,-0.638925,-0.454846,-2.840094,0.851124,0.944205,0.655093,1.078506,-1.758990,3.489748,-2.131967,-1.586815,-0.527848,0.411354,-0.035529,-0.393873,-0.54339,-0.142673
1,-2.948525,-2.634593,0.253679,-0.792972,-2.628221,0.811251,0.362717,0.520256,1.001046,-1.600288,0.135484,-2.118128,-1.545888,-0.247694,1.054127,-0.035529,-0.393873,-0.54339,-0.142673
2,-2.904607,-2.822703,-0.817446,-0.319596,-2.719024,3.017604,3.076330,3.284417,3.179828,-1.633699,-0.088134,-2.106266,-1.581904,-0.683489,-0.481386,-0.035529,-0.082421,-0.54339,-0.142673
3,-2.548869,-2.497600,-0.862076,0.289031,-2.460742,0.399221,0.634078,0.574191,0.488488,-1.789060,0.247293,-2.215001,-1.729241,-0.605669,0.232806,-0.035529,-0.393873,-0.54339,-0.142673
4,-2.344650,-2.025279,0.595844,-0.792972,-2.026908,0.558716,0.362717,0.614642,0.587374,-1.424881,-0.647178,-2.007417,-1.385455,0.406000,-0.481386,-0.035529,1.371018,-0.54339,-0.142673


In [34]:
data_final.describe().T

,count,mean,std,min,25%,50%,75%,max
SCHEDULED_DEPARTURE,1063439.0,-1.757517e-16,1.0,-3.220818,-0.763594,0.116966,0.806482,1.957139
DEPARTURE_TIME,1063439.0,-2.253555e-16,1.0,-3.117136,-0.757580,0.160479,0.812731,1.788042
DEPARTURE_DELAY,1063439.0,-3.324742e-17,1.0,-1.293501,-0.564541,-0.267006,0.223926,28.757504
TAXI_OUT,1063439.0,1.325621e-17,1.0,-1.333974,-0.590097,-0.319596,0.221406,13.814077
WHEELS_OFF,1063439.0,2.197965e-16,1.0,-3.122590,-0.681016,0.144277,0.789982,1.718184
SCHEDULED_TIME,1063439.0,-6.975545e-18,1.0,-1.660929,-0.757121,-0.238761,0.425803,7.642972
ELAPSED_TIME,1063439.0,-4.832101e-17,1.0,-1.691876,-0.735651,-0.231694,0.440249,7.999601
AIR_TIME,1063439.0,6.943473e-17,1.0,-1.475334,-0.747213,-0.248316,0.425870,7.720559
DISTANCE,1063439.0,5.510947e-17,1.0,-1.321121,-0.740991,-0.261395,0.389602,6.840249
WHEELS_ON,1063439.0,1.029494e-16,1.0,-2.637697,-0.564550,0.230629,0.768545,1.369941


In [35]:
data_final['Delayed'] = data_final['ARRIVAL_DELAY'].apply(lambda x : 1 if x > 15 else 0)
data_final.head()

,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,Delayed
0,-2.970484,-2.859507,-0.638925,-0.454846,-2.840094,0.851124,0.944205,0.655093,1.078506,-1.758990,3.489748,-2.131967,-1.586815,-0.527848,0.411354,-0.035529,-0.393873,-0.54339,-0.142673,0
1,-2.948525,-2.634593,0.253679,-0.792972,-2.628221,0.811251,0.362717,0.520256,1.001046,-1.600288,0.135484,-2.118128,-1.545888,-0.247694,1.054127,-0.035529,-0.393873,-0.54339,-0.142673,0
2,-2.904607,-2.822703,-0.817446,-0.319596,-2.719024,3.017604,3.076330,3.284417,3.179828,-1.633699,-0.088134,-2.106266,-1.581904,-0.683489,-0.481386,-0.035529,-0.082421,-0.54339,-0.142673,0
3,-2.548869,-2.497600,-0.862076,0.289031,-2.460742,0.399221,0.634078,0.574191,0.488488,-1.789060,0.247293,-2.215001,-1.729241,-0.605669,0.232806,-0.035529,-0.393873,-0.54339,-0.142673,0
4,-2.344650,-2.025279,0.595844,-0.792972,-2.026908,0.558716,0.362717,0.614642,0.587374,-1.424881,-0.647178,-2.007417,-1.385455,0.406000,-0.481386,-0.035529,1.371018,-0.54339,-0.142673,0


In [36]:
X = data_final.drop(columns = ['Delayed'],axis = 1)
y = data_final['Delayed']

In [37]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2 ,random_state = 42)

For Model Training for that we can use what ever the model we like : Logistic Regression,RandomForest,

In [38]:
#logistic
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)

LogisticRegression()

In [40]:
y_pred = log_reg.predict(X_test)

In [41]:
confusion_matrix(y_test,y_pred)

array([[212637,      1],
       [     3,     47]])

In [42]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    212638
           1       0.98      0.94      0.96        50

    accuracy                           1.00    212688
   macro avg       0.99      0.97      0.98    212688
weighted avg       1.00      1.00      1.00    212688



In [43]:
rand = RandomForestClassifier()
rand.fit(X_train,y_train)

RandomForestClassifier()

In [44]:
y_pred2 = rand.predict(X_test)

In [45]:
confusion_matrix(y_test,y_pred2)

array([[212638,      0],
       [     0,     50]])

In [46]:
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    212638
           1       1.00      1.00      1.00        50

    accuracy                           1.00    212688
   macro avg       1.00      1.00      1.00    212688
weighted avg       1.00      1.00      1.00    212688



In [47]:
xgb = XGBClassifier()
xgb.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [48]:
y_pred3 = xgb.predict(X_test)

In [49]:
confusion_matrix(y_test,y_pred3)

array([[212638,      0],
       [    50,      0]])

In [50]:
print(classification_report(y_test,y_pred3))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    212638
           1       0.00      0.00      0.00        50

    accuracy                           1.00    212688
   macro avg       0.50      0.50      0.50    212688
weighted avg       1.00      1.00      1.00    212688



In [51]:
import lightgbm as lgb

lgbm = lgb.LGBMClassifier()
lgbm.fit(X_train, y_train)
y_pred_lgbm = lgbm.predict(X_test)

print(classification_report(y_test, y_pred_lgbm))

[LightGBM] [Info] Number of positive: 181, number of negative: 850570
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.061128 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4433
[LightGBM] [Info] Number of data points in the train set: 850751, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000213 -> initscore=-8.455165
[LightGBM] [Info] Start training from score -8.455165
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [54]:
from catboost import CatBoostClassifier

catboost = CatBoostClassifier(verbose=0) # Set verbose to 0 to suppress output during training
catboost.fit(X_train, y_train)
y_pred_catboost = catboost.predict(X_test)

print(classification_report(y_test, y_pred_catboost))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    212638
           1       0.91      0.82      0.86        50

    accuracy                           1.00    212688
   macro avg       0.96      0.91      0.93    212688
weighted avg       1.00      1.00      1.00    212688



In [53]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.6 MB/s eta 0:00:00


Here is a summary of the performance metrics for each model based on the classification reports:

| Model              | Precision (Class 0) | Recall (Class 0) | F1-score (Class 0) | Precision (Class 1) | Recall (Class 1) | F1-score (Class 1) | Accuracy |
|--------------------|---------------------|------------------|--------------------|---------------------|------------------|--------------------|----------|
| Logistic Regression| 1.00                | 1.00             | 1.00               | 0.98                | 0.94             | 0.96               | 1.00     |
| Random Forest      | 1.00                | 1.00             | 1.00               | 1.00                | 1.00             | 1.00               | 1.00     |
| XGBoost            | 1.00                | 1.00             | 1.00               | 0.00                | 0.00             | 0.00               | 1.00     |
| LightGBM           | 1.00                | 1.00             | 1.00               | 0.00                | 0.00             | 0.00               | 1.00     |
| CatBoost           | 1.00                | 1.00             | 1.00               | 0.91                | 0.82             | 0.86               | 1.00     |

Based on these metrics, the Random Forest classifier appears to have performed the best, achieving perfect precision, recall, and f1-score for both classes on the test set. CatBoost also performed well, especially considering the imbalanced nature of the data (very few delayed flights). Logistic Regression had a slightly lower recall and f1-score for the delayed class compared to Random Forest. XGBoost and LightGBM struggled to identify the delayed flights, resulting in a precision and recall of 0.00 for class 1.

Keep in mind that due to the extreme class imbalance, models that predict '0' for all instances will still show a high overall accuracy. It's important to focus on the precision, recall, and f1-score for the minority class (class 1 - delayed flights) when evaluating these models.

In [55]:
from sklearn.ensemble import StackingClassifier

# Define the base models
estimators = [
    ('lr', log_reg),
    ('rf', rand),
    ('xgb', xgb),
    ('lgbm', lgbm),
    ('catboost', catboost)
]

# Define the stacking classifier
stk = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

# Train the stacking classifier
stk.fit(X_train, y_train)

# Make predictions
y_pred_stk = stk.predict(X_test)

# Print classification report
print(classification_report(y_test, y_pred_stk))

[LightGBM] [Info] Number of positive: 181, number of negative: 850570
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.111653 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4433
[LightGBM] [Info] Number of data points in the train set: 850751, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000213 -> initscore=-8.455165
[LightGBM] [Info] Start training from score -8.455165
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

# Task
Implement and evaluate time-series and sequential models (LSTM, GRU, or TCN) for predicting flight delays, compare their performance with previously trained models (Logistic Regression, Random Forest, XGBoost, LightGBM, CatBoost, and a Stacking ensemble), and identify the best-performing model.

## Prepare data for sequential modeling

### Subtask:
Restructure the dataset to create sequences of flight events and their delays. This might involve grouping data by airport and time.


**Reasoning**:
Select relevant columns and sort the data by time to prepare for sequence creation.



In [56]:
relevant_columns = ['YEAR', 'MONTH', 'DAY', 'AIRLINE', 'FLIGHT_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']
data_seq = data[relevant_columns].copy()

data_seq = data_seq.sort_values(by=['YEAR', 'MONTH', 'DAY', 'SCHEDULED_DEPARTURE'])
display(data_seq.head())

,YEAR,MONTH,DAY,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,ARRIVAL_DELAY
0,2015,1,1,AS,98,ANC,SEA,5,-11.0,-22.0
1,2015,1,1,AA,2336,LAX,PBI,10,-8.0,-9.0
2,2015,1,1,US,840,SFO,CLT,20,-2.0,5.0
3,2015,1,1,AA,258,LAX,MIA,20,-5.0,-9.0
4,2015,1,1,AS,135,SEA,ANC,25,-1.0,-21.0


**Reasoning**:
Create the 'Delayed' target variable and handle missing values in the relevant columns before grouping and creating sequences.



In [57]:
data_seq['Delayed'] = data_seq['ARRIVAL_DELAY'].apply(lambda x : 1 if x > 15 else 0)

# Fill missing values in relevant columns with a placeholder or a suitable imputation strategy
# Using -1 for delays and 0 for binary flags as a simple approach for this step.
# A more sophisticated imputation might be needed depending on the sequential model.
data_seq['DEPARTURE_DELAY'] = data_seq['DEPARTURE_DELAY'].fillna(0)
data_seq['ARRIVAL_DELAY'] = data_seq['ARRIVAL_DELAY'].fillna(0)


# Check for remaining missing values in the selected columns
display(data_seq[relevant_columns + ['Delayed']].isnull().sum())

,0
YEAR,0
MONTH,0
DAY,0
AIRLINE,0
FLIGHT_NUMBER,0
ORIGIN_AIRPORT,0
DESTINATION_AIRPORT,0
SCHEDULED_DEPARTURE,0
DEPARTURE_DELAY,0
ARRIVAL_DELAY,0


**Reasoning**:
Group the sorted data by identifiers and process each group to create sequences of a fixed length. This requires iterating through the groups and extracting subsequences.



In [ ]:
sequence_length = 10 # Define a fixed sequence length

sequences = []
targets = []

# Group by flight path
grouped = data_seq.groupby(['AIRLINE', 'FLIGHT_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'])

for name, group in grouped:
    # We need at least `sequence_length` data points to create one sequence
    if len(group) >= sequence_length:
        # Iterate through the group to create sequences and corresponding targets
        for i in range(len(group) - sequence_length + 1):
            # Select a sequence of the defined length
            sequence = group.iloc[i:i + sequence_length]

            # Extract features for the sequence (excluding identifiers and the final target)
            # Keep 'SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY' for the sequence
            sequence_features = sequence[['SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']].values

            # The target is the 'Delayed' status of the *last* flight in the sequence
            target = sequence['Delayed'].iloc[-1]

            sequences.append(sequence_features)
            targets.append(target)

# Convert the lists to NumPy arrays
sequences = np.array(sequences)
targets = np.array(targets)

print(f"Created {len(sequences)} sequences with shape {sequences.shape}")
print(f"Created {len(targets)} targets with shape {targets.shape}")